In [184]:
import mne
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams.update({'font.size': 15})
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from mne.decoding import Vectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from mne.decoding import UnsupervisedSpatialFilter
from sklearn.decomposition import PCA, FastICA

from sklearn.metrics import confusion_matrix
import itertools

import os

In [185]:
"""
GOAL: Use more data for language and non-language (not restrited to visual)
      Optimize RandomForest paramter using GridSearchCV
      Keep reducing channels with lowest feature importance for each classification until score
          not improve.
      PCA the remaining channels than classify and see results
      Plot the feature importance of top 10% consistent channels across folds
      OPTIONAL: Do shotgun approach.
      
"""

'\nGOAL: Use more data for language and non-language (not restrited to visual)\n      Optimize RandomForest paramter using GridSearchCV\n      Keep reducing channels with lowest feature importance for each classification until score\n          not improve.\n      PCA the remaining channels than classify and see results\n      Plot the feature importance of top 10% consistent channels across folds\n      OPTIONAL: Do shotgun approach.\n      \n'

In [186]:
"""
classify function
"""

def classifier(features,target):
    clf = RandomForestClassifier(n_estimators=100,random_state=0)
    #clf = LogisticRegression()
    cv = StratifiedKFold(n_splits=5,shuffle=True)
    scores = []
    
    for train,test in cv.split(features,target):
        preds = []
        X_train = features[train]; y_train = target[train] 
        X_test = features[test];   y_test = target[test]

    

        twoD_X_train = Vectorizer().fit_transform(X_train)
        #print('original shape of training set',(X_train.shape))
        #print('Vectorized shape of training set',(twoD_X_train.shape))
        clf.fit(twoD_X_train,y_train)

        #print(clf.feature_importances_.shape)
        twoD_X_test = Vectorizer().fit_transform(X_test)
        #print('original shape of test set',(X_test.shape))
        #print('Vectorized shape of test set',(twoD_X_test.shape))
        
        preds.append(clf.predict(twoD_X_test))
        #print('shape of y_test is: ',np.array(y_test).shape)
        #print(np.array(preds).shape)
        d = np.transpose(preds).ravel()
        #print('shape of pred is: ',d.shape)
        
        print('Accuracy on training set {:.3f}'.format(clf.score(twoD_X_train,y_train)))
        print('Accuracy on test set {:.3f}'.format(clf.score(twoD_X_test,y_test)))
        scores.append(clf.score(twoD_X_test,y_test))
        
        target_names = ['lang', 'non-lang']

        
        report = classification_report(y_test, d, target_names=target_names)
        print(report)
        
    print('standard deviation of scores: ',np.std(scores))    
    return np.mean(scores)

In [194]:
"""
Reorganize features importance array by sum per three?

"""
def importantChannelFinder(features,target):
    forest = RandomForestClassifier(n_estimators=100,random_state=0)
    #clf = LogisticRegression()
    #cv = StratifiedKFold(n_splits=1,shuffle=True)
    scores = []
    """creating channel dictionary"""
    ch_dict = {}
    for i in range(0,125):
        ch_dict[i] = 0
    #print(ch_dict)    
    X_train,X_test,y_train,y_test = train_test_split(features,target,test_size=0.25, random_state=42)
        
    twoD_X_train = Vectorizer().fit_transform(X_train)
    forest.fit(twoD_X_train,y_train)

    """manipulate array to 125 features"""
    impt = []
    for e in forest.feature_importances_:
        impt.append(e)

    print("shape of important feat array: ",np.array(impt).shape)
#         temp = []
#         time = binary_epoch.get_data().shape[2]
#         channel_time = binary_epoch.get_data().shape[1] * binary_epoch.get_data().shape[2]
#         for i in range(0,channel_time,time):
#             #print(i)
#             hold = []
#             for j in range(i,i+time):
#                 hold.append(impt[j])

#             temp.append(np.sum(hold))    

#         #print("new shape of important feat array: ",np.array(temp).shape) 
#         a = np.array(temp)
#         ind = np.argpartition(a, -20)[-20:]  #top ten
#         print("top twenty channels: ",ind)
#         print('top twenty channels importances: ',a[ind])
        
#         for i in ind:
#             value = ch_dict[i]
#             ch_dict[i] = value + 1 
        
        
            
            
# #         twoD_X_test = Vectorizer().fit_transform(X_test)
        
        
# #         preds.append(forest.predict(twoD_X_test))
        
# #         d = np.transpose(preds).ravel()
        
# #         scores.append(forest.score(twoD_X_test,y_test))
        
# #         target_names = ['lang', 'non-lang']
# #         report = classification_report(y_test, d, target_names=target_names)
#         #print(report)
        
#     for kk in ch_dict:
#         v = master[kk]
#         master[kk] = v + ch_dict[kk]
    
#     hold = [] #keys
#     hold2 = [] #values
#     for key in ch_dict:
#         if ch_dict[key] >= 5:
#             hold.append(key)
#             hold2.append(ch_dict[key])
#     print('importanct channels shape: ',np.array(hold).shape)        
#     plt.bar(hold,hold2) 
#     hold3 = []
#     for k in hold:
#         if binary_epoch.ch_names[k] != 'STI 014':
#             hold3.append(binary_epoch.ch_names[k])

#     print(hold3)
    

In [ ]:
"""
Filter Epoch data for language vs non-language
"""



stim_combinations = {
    (5,6,4): "AALL",
    (17,18,16) :"AALL",
    
    (11,6,10):"AALN",
    (23,18,22):"AALN",
    
    (5,12,4) :"AANL",
    (17,24,16):"AANL",
    
    (11,12,10) : "AANN",
    (23,24,22): "AANN",
    
    (2,6,1):"AVLL",
    (14,18,13):"AVLL",
    
    (8,12,7):"AVNN",
    (20,24,19):"AVNN",
    
    (5,3,4) :"VALL",
    (17,15,16):"VALL",
    
    (11,9,10) :"VANN",
    (23,21,22):"VANN",
    
    (2,3,1) :"VVLL",
    (14,15,13):"VVLL",
    
    (8,3,7):"VVLN",
    (20,15,19):"VVLN",
    
    (2,9,1) :"VVNL",
    (14,21,13):"VVNL",
    
    (8,9,7) :"VVNN",
    (20,21,19):"VVNN",
    
    (14,21,4):"whatever"
    
}

"""
Function detects stim combos and changes their event id to 100 for language and 200 for non-l
"""
def find_stim_combo(epochObj,stimCombos1,stimCombos2):
    events = epochObj.events
    numOfEvents = epochObj.events.shape[0]
    #print('shape of events: ',numOfEvents)
    
    events_new = []
    combo = []
    
    for i in range(0,numOfEvents,3):
        temp = []
        #print('three')
        for h in range(i,i+3):
            #print(events[h][-1])
            combo.append(events[h])
            
        combo_tuple = (combo[0][-1],combo[1][-1],combo[2][-1])   
        #print(combo_tuple)
        for k in stimCombos1:
            if k == stim_combinations[combo_tuple]:
                #print('found combo')
                combo[0][-1] = 100
                combo[1][-1] = 100
                combo[2][-1] = 100
                temp.append(combo[0])
                temp.append(combo[1])
                temp.append(combo[2])
        
        for j in stimCombos2:
            if j == stim_combinations[combo_tuple]:
                #print('found combo')
                combo[0][-1] = 200
                combo[1][-1] = 200
                combo[2][-1] = 200
                temp.append(combo[0])
                temp.append(combo[1])
                temp.append(combo[2])        
        #print(temp)
        for p in temp:
            events_new.append(p)
        combo = []
    print("total events b4 filtering: ",len(events))    
    print("total events aftr filtering: ",len(events_new)) 
    return events_new

In [188]:
def do(epoch):
    #epoch = mne.read_epochs(p,preload=True)
    epoch.drop_channels(ch_names=[ 'LL4', 'L12'])
    epoch.drop_channels(ch_names=[ 'Nasium', 'VEOG'])
    print('dropped channels LL4,L12,NASIUM,VEOG')
    
    epoch_copy = epoch.copy()
    
    languageStimCodes = ['AALL','AVLL','VVLL']
    nonLanguageStimCodes = ['AANN','AVNN','VVNN']
    
    find_stim_combo(epoch_copy,languageStimCodes,nonLanguageStimCodes)
    binary_epoch = epoch_copy[(epoch_copy.events[:,-1]==100) | (epoch_copy.events[:,-1]==200)].copy()
    print('shape of epoch after filtered events:',binary_epoch.get_data().shape)
    
    """
    Classify filtered data and check results
    """
    features = binary_epoch.get_data()
    print('shape of features:',features.shape)
    target = binary_epoch.events[:,-1]
    print('shape of target:',target.shape)
    
    #print("mean classification score", classifier(features,target))
    
    importantChannelFinder(features,target)
    

    

In [189]:
def SubjFolderSearch(path):
    subjFolders = os.listdir(path)
    AllFiles = []
    for i in subjFolders:
        if '.DS_Store' not in i:
            AllFiles.append(os.path.join(path,i))
    
    
    return AllFiles

In [190]:
path = '/Volumes/flash64/fifFiles'
AllSetFiles = SubjFolderSearch(path)
#print(AllSetFiles)

epo = mne.read_epochs(AllSetFiles[2])


# master = {}
# for i in range(0,125):
#     master[i] = 0
    
# def find_mod3files(fileArray):
#     #good_paths = []
#     for file in fileArray:
#         epo = mne.read_epochs(file)
#         if epo.get_data().shape[0] % 3 == 0:
#             do(epo)
#             break;

# find_mod3files(AllSetFiles)

# h1 = []
# h2 = []
# for k in master:
#     h1.append(k)
#     h2.append(master[k])
    
# plt.bar(h1,h2)   

Reading /Volumes/flash64/fifFiles/20140205_1114_epo.fif ...
    Found the data of interest:
        t =       0.00 ...     500.00 ms
        0 CTF compensation matrices available
1908 matching events found
Applying baseline correction (mode: mean)
1908 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated


In [195]:
epo_copy = epo.copy();
do(epo_copy)

dropped channels LL4,L12,NASIUM,VEOG
total events b4 filtering:  1908
total events aftr filtering:  957
shape of epoch after filtered events: (957, 125, 257)
shape of features: (957, 125, 257)
shape of target: (957,)
shape of important feat array:  (32125,)


In [ ]:
testing = 4